In [1]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import CityscapesInstanceEvaluator, verify_results

from swint.config import add_swint_config

from detectron2.solver.build import maybe_add_gradient_clipping, get_default_optimizer_params
from detectron2.modeling import GeneralizedRCNNWithTTA

import torch

/home/aiteam/anaconda3/envs/pl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup(args):
  """
  Create configs and perform basic setups.
  """
  cfg = get_cfg()
  add_swint_config(cfg)
  cfg.merge_from_file(args.config_file)
  cfg.merge_from_list(args.opts)
  cfg.freeze()
  default_setup(cfg, args)
  return cfg

In [3]:
cfg = get_cfg()
add_swint_config(cfg)
cfg.merge_from_file('/home/aiteam/tykim/mylab/swin/configs/swint/faster_rcnn_swint_T_FPN_3x_.yaml')

In [4]:
from detectron2.modeling import build_backbone

model = build_backbone(cfg)

/home/aiteam/anaconda3/envs/pl/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
model

FPN(
  (fpn_lateral2): Conv2d(96, 256, kernel_size=(1, 1), stride=(1, 1))
  (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fpn_lateral3): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1))
  (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fpn_lateral4): Conv2d(384, 256, kernel_size=(1, 1), stride=(1, 1))
  (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fpn_lateral5): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
  (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (top_block): LastLevelMaxPool()
  (bottom_up): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): Swi

In [5]:
class Trainer(DefaultTrainer):
  @classmethod
  def build_optimizers(cls, cfg, model):
    params = get_default_optimizer_params(model,
                                          base_lr=cfg.SOLVER.BASE_LR,
                                          weight_decay=cfg.SOLVER.WEIGHT_DECAY,
                                          weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
                                          bias_lr_factor=cfg.SOLVER.WEIGHT_DECAY_BIAS,)
    def maybe_add_full_model_gradient_clipping(optim):  # optim: the optimizer class
      clip_norm_val = cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE
      enable = (cfg.SOLVER.CLIP_GRADIENTS.ENABLED and
                cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE == 'full_model' and
                clip_norm_val > 0.0)

      class FullModelGradientClippingOptimizer(optim):
        def step(self, closure=None):
          all_params = itertools.chain(*[x["params"] for x in self.param_groups])
          torch.nn.utils.clip_grad_norm_(all_params, clip_norm_val)
          super().step(closure=closure)


      return FullModelGradientClippingOptimizer if enable else optim

    optimizer_type = cfg.SOLVER.OPTIMIZER
    if optimizer_type == 'SGD':
      optimizer = maybe_add_gradient_clipping(cfg, torch.optim.SGD)(params,
                                                               cfg.SOLVER.BASE_LR,
                                                               momentum=cfg.SOLVER.MOMENTUM,
                                                               nesterov=cfg.SOLVER.NESTEROV,
                                                               weight_decay=cfg.SOLVER.WEIGHT_DECAY,)
    elif optimizer_type == 'AdamW':
      optimizer = maybe_add_full_model_gradient_clipping(torch.optim.AdamW)(params,
                                                                            cfg.SOLVER.BASE_LR,
                                                                            betas=(0.9, 0.99),
                                                                            weight_decay=cfg.SOLVER.WEIGHT_DECAY)
    else:
      raise NotImplementedError(f'no optimizer type {optimizer_type}')
    return optimizer

      

In [9]:
model = Trainer.build_model(cfg)

In [ ]:
from detec

In [ ]:
# Infernece
cfg.MODEL.WEIGHTS = os.p